In [1]:
import itertools
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, StorageLevel
import sys

In [2]:
conf = SparkConf()
# conf.set("spark.driver.memory", "4g")
# conf.set("spark.executor.memory", "4g")
conf.setMaster('local[8]')
conf.setAppName('Assignment_2')
sc = SparkContext.getOrCreate(conf)

In [3]:
def get_all_candidates(bucket):
    all_candidates = []
    singleton_count_map = {}
    candidate_singletons = []
    pair_count_map = {}
    candidate_pairs = []

    for basket in bucket:
        basket.sort()
        for item in basket:
            if item not in singleton_count_map:
                singleton_count_map[item] = 0
            singleton_count_map[item] += 1

        for pair in itertools.combinations(basket, 2):
            if frozenset(pair) not in pair_count_map:
                pair_count_map[frozenset(pair)] = 0
            pair_count_map[frozenset(pair)] += 1

    for item in singleton_count_map:
        if singleton_count_map[item] >= partition_support:
            candidate_singletons.append((item))

    all_candidates.append([1, candidate_singletons])

    for pair in itertools.combinations(candidate_singletons, 2):
        pair = frozenset(pair)
        if pair in pair_count_map and pair_count_map[pair] >= partition_support:
            candidate_pairs.append(pair)

    all_candidates.append([2, candidate_pairs])

    set_size = 3
    previous_candidates = candidate_pairs

    while 1:
        current_candidates = []
        for i, subcand_1 in enumerate(previous_candidates):
            for subcand_2 in previous_candidates[i + 1:]:
                superset_cand = subcand_1.union(subcand_2)

                if len(superset_cand) == set_size and superset_cand not in current_candidates:
                    current_candidates.append(superset_cand)

        supported_candidates = []
        for cand_item_set in current_candidates:
            part_sup_cand = 0
            for basket in bucket:
                if cand_item_set.issubset(set(basket)):
                    part_sup_cand += 1

                    if part_sup_cand >= partition_support:
                        supported_candidates.append(cand_item_set)

        for cand_item_set in supported_candidates:
            for subset in itertools.combinations(cand_item_set, set_size - 1):
                subset = set(subset)
                if subset not in all_candidates[set_size - 2][1]:
                    supported_candidates.remove(cand_item_set)

        all_candidates.append((set_size, supported_candidates))

        previous_candidates = supported_candidates
        set_size += 1

        if not previous_candidates:
            break

    resulting_candidates = []
    for cand_set in all_candidates:
        if cand_set[0] == 1:
            candidates = [tuple((x,)) for x in cand_set[1]]
        else:
            candidates = [tuple(sorted(x)) for x in cand_set[1]]
        yield ((cand_set[0], candidates))


In [33]:
def get_original_frequent_sets(bucket, all_candidates):
    for candidate_set in all_candidates:
        for candidate in candidate_set:
            for basket in bucket:
                if set(candidate).issubset(basket):
                    yield(candidate, 1)
    

In [5]:
import time

start_time = time.time()
threshold = 1
case = '1'
support = 4

In [17]:
data_path = 'asnlib/publicdata/'
# data_path = ''

data = sc.textFile(data_path + 'small1.csv').map(lambda x: x.split(',')).map(lambda x: (x[0], x[1]))
header = data.first()
raw_data = data.filter(lambda x: x != header)


if case == '2':
    raw_data = raw_data.map(lambda x: (x[1], x[0]))

baskets = raw_data.groupByKey().map(lambda x: (list(set(x[1])))).filter(lambda x: len(x) >= threshold)
# print(baskets.collect())
# baskets.persist(StorageLevel.DISK_ONLY)
count_baskets = baskets.count()

In [ ]:
count_baskets

In [ ]:
parl_baskets = sc.parallelize(user_baskets.collect(),2)

In [15]:
baskets = parl_baskets.values()

NameError: name 'parl_baskets' is not defined

In [18]:
baskets.take(2)

[['102', '100', '101', '98'], ['102', '97', '101', '99', '103']]

In [19]:
partition_support = support/2

In [20]:
candidate_item_sets = baskets.mapPartitions(lambda x: get_all_candidates(list(x)))

In [21]:
candidate_item_sets_2 = candidate_item_sets.reduceByKey(lambda a,b: a+b)

In [22]:
candidate_item_sets_2.count()

6

In [ ]:
candidate_item_sets_2.collect()

In [23]:
candidate_item_sets_3 = candidate_item_sets_2.map(lambda x: list(set(x[1])))

In [ ]:
candidate_item_sets_3.collect()

In [24]:
candidate_item_sets_4 = candidate_item_sets_3.filter(lambda x: len(x)>0)

In [ ]:
candidate_item_sets_4.collect()

In [25]:
candidate_item_sets_5 = candidate_item_sets_4.sortBy(lambda x: len(x[0]))

In [ ]:
candidate_item_sets_5.collect()

In [34]:
cands = candidate_item_sets_5.collect()
original_freq_itemsets = baskets.mapPartitions(lambda x: get_original_frequent_sets(list(x), cands))

In [35]:
original_freq_itemsets.collect()

[(('100',), 1),
 (('100',), 1),
 (('100',), 1),
 (('102',), 1),
 (('102',), 1),
 (('102',), 1),
 (('102',), 1),
 (('102',), 1),
 (('103',), 1),
 (('103',), 1),
 (('105',), 1),
 (('98',), 1),
 (('98',), 1),
 (('98',), 1),
 (('98',), 1),
 (('98',), 1),
 (('98',), 1),
 (('98',), 1),
 (('97',), 1),
 (('97',), 1),
 (('97',), 1),
 (('97',), 1),
 (('97',), 1),
 (('97',), 1),
 (('97',), 1),
 (('101',), 1),
 (('101',), 1),
 (('101',), 1),
 (('101',), 1),
 (('99',), 1),
 (('99',), 1),
 (('99',), 1),
 (('99',), 1),
 (('99',), 1),
 (('103', '98'), 1),
 (('102', '98'), 1),
 (('102', '98'), 1),
 (('102', '98'), 1),
 (('102', '98'), 1),
 (('98', '99'), 1),
 (('98', '99'), 1),
 (('98', '99'), 1),
 (('100', '101'), 1),
 (('100', '101'), 1),
 (('100', '101'), 1),
 (('100', '102'), 1),
 (('100', '102'), 1),
 (('102', '103'), 1),
 (('102', '103'), 1),
 (('101', '97'), 1),
 (('102', '99'), 1),
 (('102', '99'), 1),
 (('105', '98'), 1),
 (('102', '97'), 1),
 (('102', '97'), 1),
 (('102', '97'), 1),
 (('101',

In [36]:
a_1 = original_freq_itemsets.reduceByKey(lambda a,b: a+b)

In [37]:
a_2 = a_1.filter(lambda x: x[1]>=support)

In [38]:
a_3 = a_2.map(lambda x: (len(x[0]), [x[0]]))

In [54]:
a_4 = a_3.groupByKey().map(lambda x: (x[0], list(x[1])))

In [55]:
a_5 = a_4.sortBy(lambda x: x[0])

In [56]:
a_5.collect()

[(1,
  [[('103',)],
   [('97',)],
   [('101',)],
   [('99',)],
   [('100',)],
   [('102',)],
   [('98',)]]),
 (2,
  [[('102', '98')],
   [('101', '97')],
   [('103', '99')],
   [('97', '99')],
   [('100', '98')],
   [('101', '99')],
   [('98', '99')],
   [('100', '101')],
   [('102', '103')],
   [('102', '99')],
   [('102', '97')],
   [('101', '102')],
   [('97', '98')],
   [('101', '98')]]),
 (3,
  [[('101', '97', '99')],
   [('100', '101', '98')],
   [('97', '98', '99')],
   [('102', '103', '99')]])]